In [1]:
import pandas as pd
#from collections import defaultdict
#import csv
import re 
from string import punctuation
from nltk import word_tokenize, wordpunct_tokenize
from collections import Counter
import main
from importlib import reload
import numpy as np

In [2]:
filename="RS_2011-01_1000"
outputname='filtered_'+filename[3:10]
sub_list= set(['reddit.com','leagueoflegends', 'gaming', 'DestinyTheGame', 'DotA2', 'ContestofChampions', 'StarWarsBattlefront', 'Overwatch', 'WWII', 'hearthstone', 'wow', 'heroesofthestorm', 'destiny2', 'darksouls3', 'fallout', 'SuicideWatch', 'depression', 'OCD', 'dpdr', 'proED', 'Anxiety', 'BPD', 'socialanxiety', 'mentalhealth', 'ADHD', 'bipolar', 'buildapc', 'techsupport', 'buildapcforme', 'hacker', 'SuggestALaptop', 'hardwareswap', 'laptops', 'computers', 'pcmasterrace', 'relationshps', 'relationship_advice', 'breakups', 'dating_advice', 'LongDistance', 'polyamory', 'wemetonline', 'MDMA', 'Drugs', 'trees', 'opiates', 'LSD', 'tifu', 'r4r', 'AskReddit', 'reddit.com', 'tipofmytongue', 'Life', 'Advice', 'jobs', 'teenagers', 'HomeImprovement', 'redditinreddit', 'FIFA', 'nba', 'hockey', 'nfl', 'mls', 'baseball', 'BokuNoHeroAcademia', 'anime', 'movies', 'StrangerThings'])
minwords=100

In [3]:
alldata = pd.read_json(filename, lines=True)
if sub_list!=set():
    alldata=alldata[alldata.subreddit.isin(sub_list)]


In [4]:
alldata

,approved_by,author,author_flair_css_class,author_flair_text,banned_by,clicked,created,created_utc,distinguished,domain,...,saved,score,selftext,selftext_html,subreddit,subreddit_id,thumbnail,title,ups,url
9,NaN,thenewguy729,None,None,NaN,False,1293944352,1293944352,None,self.gaming,...,False,1,So I bought Amnesia on the Steam sale the othe...,"&lt;!-- SC_OFF --&gt;&lt;div class=""md""&gt;&lt...",gaming,t5_2qh03,self,Can You Run It lied to me. Help?,1,http://www.reddit.com/r/gaming/comments/eut3s/...
12,NaN,frnak,None,None,NaN,False,1293944322,1293944322,None,self.AskReddit,...,False,2,"People just hang up, like both parties know ex...","&lt;!-- SC_OFF --&gt;&lt;div class=""md""&gt;&lt...",AskReddit,t5_2qh1i,self,Why can't phone calls on TV and in movies end ...,4,http://www.reddit.com/r/AskReddit/comments/eut...
18,NaN,SteveMarcus,None,None,NaN,False,1293944292,1293944292,None,self.AskReddit,...,False,2,So I deliver(ed?) pizza for a local establishm...,"&lt;!-- SC_OFF --&gt;&lt;div class=""md""&gt;&lt...",AskReddit,t5_2qh1i,self,I was impersonally fired from my pizza deliver...,2,http://www.reddit.com/r/AskReddit/comments/eut...
19,NaN,[deleted],None,None,NaN,False,1293944269,1293944269,None,imgur.com,...,False,3,,None,trees,t5_2r9vp,default,I am at [5] and I just noticed I was eating...,8,http://imgur.com/l0pjn
20,NaN,ooglytoop7272,None,None,NaN,False,1293944267,1293944267,None,self.AskReddit,...,False,0,They've been saying the script is half done fo...,"&lt;!-- SC_OFF --&gt;&lt;div class=""md""&gt;&lt...",AskReddit,t5_2qh1i,self,Does anybody know the status on the Arrested D...,5,http://www.reddit.com/r/AskReddit/comments/eut...
22,NaN,slh1191,None,None,NaN,False,1293944264,1293944264,None,hollyhotty.com,...,False,0,,None,reddit.com,t5_6,default,kelly brooks,0,http://www.hollyhotty.com/?p=1837
24,NaN,solton,None,None,NaN,False,1293944261,1293944261,None,themallblog.com,...,False,1,,None,reddit.com,t5_6,default,PLR Profits,1,http://www.themallblog.com/profit-plr-0
28,NaN,[deleted],None,None,NaN,False,1293944254,1293944254,None,gaming.stackexchange.com,...,False,1,,None,gaming,t5_2qh03,default,Xbox Kinect user is experiencing vision proble...,1,http://gaming.stackexchange.com/questions/1371...
34,NaN,kcocampo,None,None,NaN,False,1293944209,1293944209,None,self.reddit.com,...,False,1,,None,reddit.com,t5_6,default,"Once Learned, Video Editing Can Be Easy",1,http://www.reddit.com/r/reddit.com/comments/eu...
35,NaN,[deleted],None,None,NaN,False,1293944208,1293944208,None,youtubedoubler.com,...,False,1,,None,reddit.com,t5_6,default,"Dance with Manbungo ,The African Warrior,every...",1,http://youtubedoubler.com/?video1=s-MqaCCbDco


In [5]:
selftext= alldata[alldata['is_self']] 	\
    .loc[:,['id','subreddit','selftext']]
selftext


,id,subreddit,selftext
9,eut3s,gaming,So I bought Amnesia on the Steam sale the othe...
12,eut3p,AskReddit,"People just hang up, like both parties know ex..."
18,eut3i,AskReddit,So I deliver(ed?) pizza for a local establishm...
20,eut3g,AskReddit,They've been saying the script is half done fo...
34,eut30,reddit.com,
36,eut2y,gaming,Or does anyone know where someone has analyzed...
37,eut2x,reddit.com,
42,eut2s,AskReddit,I'd rather not download a torrent if at all po...
53,eut2h,AskReddit,I could've never beaten Majora's Mask without ...
55,eut2f,reddit.com,


In [6]:
selftext['selftext']=selftext.transform({'selftext': lambda x: main.tokenize(x)})#, lambda x: tokenize(x)[1]] })

selftext[['counter', 'wordcount']] = selftext['selftext'].apply(pd.Series)



In [7]:
filtered= selftext[selftext['wordcount'] >=minwords] \
        .loc[:,['id','subreddit','counter', 'wordcount']]


In [8]:
filtered

,id,subreddit,counter,wordcount
9,eut3s,gaming,"{'so': 1, 'i': 9, 'bought': 3, 'amnesia': 3, '...",182
18,eut3i,AskReddit,"{'so': 1, 'i': 10, 'deliver': 1, '(': 5, 'ed':...",241
57,eut2d,AskReddit,"{'okay': 1, 'so': 2, 'my': 2, 'cousins': 1, 'a...",152
86,eut1k,AskReddit,"{'google': 1, 'queries': 1, 'aside': 1, ',': 5...",104
186,eusyq,AskReddit,"{'so': 5, 'my': 15, 'sister': 10, '(': 16, 'i'...",624
190,eusym,reddit.com,"{'katey': 4, 'says': 7, ':': 7, 'i': 5, 'bough...",105
201,eusyb,AskReddit,"{'my': 1, 'wife': 1, 'is': 3, 'a': 3, 'talente...",127
223,eusxn,AskReddit,"{'i': 2, 'am': 1, 'trying': 2, 'to': 6, 'set':...",114
230,eusxf,gaming,"{'i': 5, ''m': 2, 'trying': 2, 'to': 4, 'get':...",145
265,euswf,reddit.com,"{'welcome': 1, 'to': 4, 'www.valentinediamondc...",146


# Dictionaries

In [9]:
main.WordCollection.wcs_from_file("wordCollections.dic")
absolutist = ['absolutely', 'all', 'always', 'complete', 'competely','constant', 'constantly', 'definitely', 'entire', 'ever', 'every', 'everyone', 'everything', 'full', 'must', 'never', 'nothing', 'totally','whole']
main.WordCollection(0,'absolutist', absolutist)

In [10]:
reload(main)
abscounts = main.calculatePosts2(filtered)
abscounts

word so
no match so
[]
word i
no match i
[]
word bought
no match bought
[]
word amnesia
no match amnesia
[]
word on
no match on
[]
word the
no match the
[]
word steam
no match steam
[]
word sale
no match sale
[]
word other
no match other
[]
word day
no match day
[]
word ,
no match ,
[]
word thinking
no match thinking
[]
word ``
no match ``
[]
word yes
no match yes
[]
word !
no match !
[]
word time
no match time
[]
word to
no match to
[]
word scare
no match scare
[]
word crap
no match crap
[]
word out
no match out
[]
word of
no match of
[]
word myself
no match myself
[]
word .
no match .
[]
word and
no match and
[]
word hey
no match hey
[]
word look
no match look
[]
word can
no match can
[]
word you
no match you
[]
word run
no match run
[]
word it
no match it
[]
word says
no match says
[]
word my
no match my
[]
word shitty
no match shitty
[]
word computer
no match computer
[]
word passes
no match passes
[]
word with
no match with
[]
word flying
no match flying
[]
word colors
no match co

word harrass
no match harrass
[]
word had
no match had
[]
word email
no match email
[]
word telling
no match telling
[]
word how
no match how
[]
word going
no match going
[]
word be
no match be
[]
word ``
no match ``
[]
word mommy
no match mommy
[]
word ''
no match ''
[]
word much
no match much
[]
word better
no match better
[]
word 'll
no match 'll
[]
word it
no match it
[]
word some
no match some
[]
word psycho
no match psycho
[]
word woman
no match woman
[]
word with
no match with
[]
word 5
no match 5
[]
word own
no match own
[]
word made
no match made
[]
word physical
no match physical
[]
word threats
no match threats
[]
word text
no match text
[]
word messages
no match messages
[]
word emails
no match emails
[]
word were
no match were
[]
word erased
no match erased
[]
word by
no match by
[]
word brother-in-law
no match brother-in-law
[]
word closed
no match closed
[]
word out
no match out
[]
word their
no match their
[]
word checking
no match checking
[]
word account
no match acco

word at
no match at
[]
word moment
no match moment
[]
word and
no match and
[]
word find
no match find
[]
word a
no match a
[]
word connector
no match connector
[]
word that
no match that
[]
word would
no match would
[]
word allow
no match allow
[]
word from
no match from
[]
word go
no match go
[]
word does
no match does
[]
word anyone
no match anyone
[]
word know
no match know
[]
word of
no match of
[]
word male
no match male
[]
word female
no match female
[]
word inputs
no match inputs
[]
word output
no match output
[]
word or
no match or
[]
word some
no match some
[]
word other
no match other
[]
word work
no match work
[]
word i
no match i
[]
word 'm
no match 'm
[]
word trying
no match trying
[]
word to
no match to
[]
word get
no match get
[]
word my
no match my
[]
word little
no match little
[]
word brother
no match brother
[]
word interested
no match interested
[]
word in
no match in
[]
word some
no match some
[]
word computer
no match computer
[]
word gaming
no match gaming
[]
wo

no match thing
[]
word find
no match find
[]
word is
no match is
[]
word 'device
no match 'device
[]
word status
no match status
[]
word '
no match '
[]
word refer
no match refer
[]
word keeps
no match keeps
[]
word changing
no match changing
[]
word alternates
no match alternates
[]
word between
no match between
[]
word 'this
no match 'this
[]
word can
no match can
[]
word not
no match not
[]
word start
no match start
[]
word 'no
no match 'no
[]
word drivers
no match drivers
[]
word were
no match were
[]
word found
no match found
[]
word 'something
no match 'something
[]
word else
no match else
[]
word about
no match about
[]
word let
no match let
[]
word me
no match me
[]
word know
no match know
[]
word if
no match if
[]
word there
no match there
[]
word anything
no match anything
[]
word you
no match you
[]
word might
no match might
[]
word need
no match need
[]
word so
no match so
[]
word i
no match i
[]
word want
no match want
[]
word to
no match to
[]
word learn
no match learn
[]

[]
word nowhere
no match nowhere
[]
word shit
no match shit
[]
word that
no match that
[]
word happening
no match happening
[]
word out
no match out
[]
word there
no match there
[]
word *heinous*
no match *heinous*
[]
word generation
no match generation
[]
word doing
no match doing
[]
word absolutely
no match absolutely
[]
word *nothing
no match *nothing
[]
word worth
no match worth
[]
word stop
no match stop
[]
word it*
no match it*
[]
word i
no match i
[]
word for
no match for
[]
word one
no match one
[]
word am
no match am
[]
word sick
no match sick
[]
word tired
no match tired
[]
word sitting
no match sitting
[]
word my
no match my
[]
word screen
no match screen
[]
word just
no match just
[]
word being
no match being
[]
word appalled
no match appalled
[]
word want
no match want
[]
word get
no match get
[]
word actually
no match actually
[]
word do
no match do
[]
word something
no match something
[]
word feel
no match feel
[]
word like
no match like
[]
word you
no match you
[]
word 

...

word 2008
no match 2008
[]
word rapid
no match rapid
[]
word and
no match and
[]
word overwhelming
no match overwhelming
[]
word series
no match series
[]
word events
no match events
[]
word forced
no match forced
[]
word me
no match me
[]
word realize
no match realize
[]
word that
no match that
[]
word if
no match if
[]
word there
no match there
[]
word god
no match god
[]
word he
no match he
[]
word royal
no match royal
[]
word jackass
no match jackass
[]
word unworth
no match unworth
[]
word any
no match any
[]
word man
no match man
[]
word 's
no match 's
[]
word love
no match love
[]
word or
no match or
[]
word obedience
no match obedience
[]
word this
no match this
[]
word sent
no match sent
[]
word spiraling
no match spiraling
[]
word into
no match into
[]
word depression
no match depression
[]
word eventually
no match eventually
[]
word bastard
no match bastard
[]
word combination
no match combination
[]
word nihilism
no match nihilism
[]
word got
no match got
[]
word job
no mat

word thank
no match thank
[]
word much
no match much
[]
word !
no match !
[]
word i
no match i
[]
word just
no match just
[]
word saw
no match saw
[]
word black
no match black
[]
word swan
no match swan
[]
word ,
no match ,
[]
word which
no match which
[]
word makes
no match makes
[]
word the
no match the
[]
word fountain
no match fountain
[]
word only
no match only
[]
word film
no match film
[]
word from
no match from
[]
word him
no match him
[]
word have
no match have
[]
word n't
no match n't
[]
word seen
no match seen
[]
word yet
no match yet
[]
word .
no match .
[]
word but
no match but
[]
word given
no match given
[]
word ending
no match ending
[]
word while
no match while
[]
word her
no match her
[]
word mind
no match mind
[]
word may
no match may
[]
word be
no match be
[]
word gone
no match gone
[]
word or
no match or
[]
word broken
no match broken
[]
word people
no match people
[]
word injured
no match injured
[]
word were
no match were
[]
word nina
no match nina
[]
word with
n

no match psychologist
[]
word specialized
no match specialized
[]
word cognitive
no match cognitive
[]
word emdr
no match emdr
[]
word six
no match six
[]
word months
no match months
[]
word after
no match after
[]
word improved
no match improved
[]
word enough
no match enough
[]
word begin
no match begin
[]
word see
no match see
[]
word psychiatrist
no match psychiatrist
[]
word we
no match we
[]
word slowly
no match slowly
[]
word which
no match which
[]
word medications
no match medications
[]
word dosages
no match dosages
[]
word finally
no match finally
[]
word turned
no match turned
[]
word corner
no match corner
[]
word have
no match have
[]
word there
no match there
[]
word know
no match know
[]
word mean
no match mean
[]
word realization
no match realization
[]
word one
no match one
[]
word sick
no match sick
[]
word under
no match under
[]
word water
no match water
[]
word pool
no match pool
[]
word everything
no match everything
[]
word distorted
no match distorted
[]
word m

[]
word they
no match they
[]
word mostly
no match mostly
[]
word been
no match been
[]
word without
no match without
[]
word support
no match support
[]
word hired
no match hired
[]
word guy
no match guy
[]
word year
no match year
[]
word according
no match according
[]
word cousin
no match cousin
[]
word he
no match he
[]
word is
no match is
[]
word very
no match very
[]
word inexperienced
no match inexperienced
[]
word the
no match the
[]
word has
no match has
[]
word zero
no match zero
[]
word servers
no match servers
[]
word implemented
no match implemented
[]
word are
no match are
[]
word looking
no match looking
[]
word for
no match for
[]
word someone
no match someone
[]
word help
no match help
[]
word build
no match build
[]
word their
no match their
[]
word infrastructure
no match infrastructure
[]
word as
no match as
[]
word well
no match well
[]
word provide
no match provide
[]
word employees
no match employees
[]
word primarily
no match primarily
[]
word use
no match use
[

,wordcount
subreddit,
AskReddit,8002
SuicideWatch,329
depression,1637
gaming,775
hockey,193
movies,100
reddit.com,1021
relationship_advice,928
techsupport,463


In [11]:
reload(main)
filtered['dictcounts']=filtered.transform({'counter': lambda x: main.getfreqs(x)})
filtered

word so
no match so
[]
word i
no match i
[]
word bought
no match bought
[]
word amnesia
no match amnesia
[]
word on
no match on
[]
word the
no match the
[]
word steam
no match steam
[]
word sale
no match sale
[]
word other
no match other
[]
word day
no match day
[]
word ,
no match ,
[]
word thinking
no match thinking
[]
word ``
no match ``
[]
word yes
no match yes
[]
word !
no match !
[]
word time
no match time
[]
word to
no match to
[]
word scare
no match scare
[]
word crap
no match crap
[]
word out
no match out
[]
word of
no match of
[]
word myself
no match myself
[]
word .
no match .
[]
word and
no match and
[]
word hey
no match hey
[]
word look
no match look
[]
word can
no match can
[]
word you
no match you
[]
word run
no match run
[]
word it
no match it
[]
word says
no match says
[]
word my
no match my
[]
word shitty
no match shitty
[]
word computer
no match computer
[]
word passes
no match passes
[]
word with
no match with
[]
word flying
no match flying
[]
word colors
no match co

word closed
no match closed
[]
word right
no match right
[]
word away
no match away
[]
word it
no match it
[]
word probably
no match probably
[]
word 2cm
no match 2cm
[]
word long
no match long
[]
word 3mm
no match 3mm
[]
word deep
no match deep
[]
word threw
no match threw
[]
word some
no match some
[]
word iodine
no match iodine
[]
word on
no match on
[]
word taped
no match taped
[]
word up
no match up
[]
word with
no match with
[]
word sterile
no match sterile
[]
word gauze
no match gauze
[]
word clothespin
no match clothespin
[]
word as
no match as
[]
word light
no match light
[]
word splint
no match splint
[]
word should
no match should
[]
word do
no match do
[]
word anything
no match anything
[]
word else
no match else
[]
word ?
no match ?
[]
word tl
no match tl
[]
word ;
no match ;
[]
word dr
no match dr
[]
word :
no match :
[]
word open
no match open
[]
word how
no match how
[]
word necessary
no match necessary
[]
word for
no match for
[]
word so
no match so
[]
word my
no match

[]
word share
no match share
[]
word will
no match will
[]
word travel
no match travel
[]
word anywhere
no match anywhere
[]
word country
no match country
[]
word for
no match for
[]
word everything
no match everything
[]
word portraits
no match portraits
[]
word art
no match art
[]
word name
no match name
[]
word it
no match it
[]
word feed
no match feed
[]
word bad
no match bad
[]
word appreciated
no match appreciated
[]
word !
no match !
[]
word i
no match i
[]
word am
no match am
[]
word trying
no match trying
[]
word to
no match to
[]
word set
no match set
[]
word up
no match up
[]
word my
no match my
[]
word ps3
no match ps3
[]
word have
no match have
[]
word sound
no match sound
[]
word going
no match going
[]
word through
no match through
[]
word computer
no match computer
[]
word speakers
no match speakers
[]
word ,
no match ,
[]
word but
no match but
[]
word they
no match they
[]
word do
no match do
[]
word n't
no match n't
[]
word an
no match an
[]
word aux
no match aux
[]
w

no match .
[]
word the
no match the
[]
word real
no match real
[]
word not
no match not
[]
word silly
no match silly
[]
word tiles
no match tiles
[]
word matching
no match matching
[]
word game
no match game
[]
word 'd
no match 'd
[]
word like
no match like
[]
word hong
no match hong
[]
word kong
no match kong
[]
word style
no match style
[]
word if
no match if
[]
word possible
no match possible
[]
word ,
no match ,
[]
word but
no match but
[]
word others
no match others
[]
word are
no match are
[]
word fine
no match fine
[]
word you
no match you
[]
word can
no match can
[]
word tell
no match tell
[]
word me
no match me
[]
word what
no match what
[]
word version
no match version
[]
word 'm
no match 'm
[]
word learning
no match learning
[]
word read
no match read
[]
word chinese
no match chinese
[]
word numbers
no match numbers
[]
word do
no match do
[]
word n't
no match n't
[]
word know
no match know
[]
word dragons
no match dragons
[]
word and
no match and
[]
word other
no match other

no match group
[]
word recent
no match recent
[]
word memory
no match memory
[]
word make
no match make
[]
word serious
no match serious
[]
word strides
no match strides
[]
word towards
no match towards
[]
word transparency
no match transparency
[]
word name
no match name
[]
word few
no match few
[]
word unjust
no match unjust
[]
word occurring
no match occurring
[]
word *at
no match *at
[]
word very
no match very
[]
word moment*
no match moment*
[]
word **we**
no match **we**
[]
word need
no match need
[]
word *do
no match *do
[]
word anything*
no match anything*
[]
word because
no match because
[]
word n't
no match n't
[]
word know
no match know
[]
word about
no match about
[]
word but
no match but
[]
word driving
no match driving
[]
word me
no match me
[]
word crazy
no match crazy
[]
word **edit**
no match **edit**
[]
word :
no match :
[]
word *holy
no match *holy
[]
word fucking
no match fucking
[]
word shit*
no match shit*
[]
word leave
no match leave
[]
word 24
no match 24
[]
wor

...

word day
no match day
[]
word difference
no match difference
[]
word break
no match break
[]
word marked
no match marked
[]
word end
no match end
[]
word disastrous
no match disastrous
[]
word first
no match first
[]
word term
no match term
[]
word college
no match college
[]
word had
no match had
[]
word been
no match been
[]
word meds
no match meds
[]
word few
no match few
[]
word days
no match days
[]
word decided
no match decided
[]
word quit
no match quit
[]
word heard
no match heard
[]
word horror
no match horror
[]
word stories
no match stories
[]
word withdrawals
no match withdrawals
[]
word did
no match did
[]
word n't
no match n't
[]
word notice
no match notice
[]
word much
no match much
[]
word out
no match out
[]
word ordinary
no match ordinary
[]
word maybe
no match maybe
[]
word that
no match that
[]
word testament
no match testament
[]
word how
no match how
[]
word doing
no match doing
[]
word anyways
no match anyways
[]
word actually
no match actually
[]
word kind
no ma

[]
word eyeballs
no match eyeballs
[]
word on
no match on
[]
word website
no match website
[]
word question
no match question
[]
word is
no match is
[]
word most
no match most
[]
word profitable
no match profitable
[]
word way
no match way
[]
word to
no match to
[]
word monetize
no match monetize
[]
word this
no match this
[]
word project
no match project
[]
word ?
no match ?
[]
word youtube
no match youtube
[]
word partnership
no match partnership
[]
word worthwhile
no match worthwhile
[]
word need
no match need
[]
word read
no match read
[]
word fine
no match fine
[]
word print
no match print
[]
word uploading
no match uploading
[]
word like
no match like
[]
word yt
no match yt
[]
word does
no match does
[]
word google
no match google
[]
word then
no match then
[]
word own
no match own
[]
word videos
no match videos
[]
word upload
no match upload
[]
word if
no match if
[]
word channel
no match channel
[]
word that
no match that
[]
word place
no match place
[]
word restrictions
no mat

no match he
[]
word was
no match was
[]
word in
no match in
[]
word love
no match love
[]
word a
no match a
[]
word woman
no match woman
[]
word of
no match of
[]
word race
no match race
[]
word 'd
no match 'd
[]
word just
no match just
[]
word disrespected
no match disrespected
[]
word but
no match but
[]
word her
no match her
[]
word family
no match family
[]
word did
no match did
[]
word n't
no match n't
[]
word accept
no match accept
[]
word him
no match him
[]
word made
no match made
[]
word bitter
no match bitter
[]
word towards
no match towards
[]
word .
no match .
[]
word so
no match so
[]
word think
no match think
[]
word some
no match some
[]
word people
no match people
[]
word being
no match being
[]
word bus
no match bus
[]
word sound
no match sound
[]
word familiar
no match familiar
[]
word anyone
no match anyone
[]
word ?
no match ?
[]
word lot
no match lot
[]
word enjoying
no match enjoying
[]
word hi
no match hi
[]
word reddit
no match reddit
[]
word !
no match !
[]
wor

no match polarized
[]
word metaphysics
no match metaphysics
[]
word argument
no match argument
[]
word no
no match no
[]
word lack
no match lack
[]
word people
no match people
[]
word discuss
no match discuss
[]
word scientific
no match scientific
[]
word portion
no match portion
[]
word quest
no match quest
[]
word unwilling
no match unwilling
[]
word issues
no match issues
[]
word outside
no match outside
[]
word particular
no match particular
[]
word framework
no match framework
[]
word likewise
no match likewise
[]
word whom
no match whom
[]
word often
no match often
[]
word well
no match well
[]
word familiar
no match familiar
[]
word things
no match things
[]
word short
no match short
[]
word talk
no match talk
[]
word life
no match life
[]
word mind
no match mind
[]
word universe
no match universe
[]
word extreme
no match extreme
[]
word if
no match if
[]
word will
no match will
[]
word provide
no match provide
[]
word list
no match list
[]
word illustrate
no match illustrate
[]

,id,subreddit,counter,wordcount,dictcounts
9,eut3s,gaming,"{'so': 1, 'i': 9, 'bought': 3, 'amnesia': 3, '...",182,{}
18,eut3i,AskReddit,"{'so': 1, 'i': 10, 'deliver': 1, '(': 5, 'ed':...",241,{}
57,eut2d,AskReddit,"{'okay': 1, 'so': 2, 'my': 2, 'cousins': 1, 'a...",152,{}
86,eut1k,AskReddit,"{'google': 1, 'queries': 1, 'aside': 1, ',': 5...",104,{}
186,eusyq,AskReddit,"{'so': 5, 'my': 15, 'sister': 10, '(': 16, 'i'...",624,{}
190,eusym,reddit.com,"{'katey': 4, 'says': 7, ':': 7, 'i': 5, 'bough...",105,{}
201,eusyb,AskReddit,"{'my': 1, 'wife': 1, 'is': 3, 'a': 3, 'talente...",127,{}
223,eusxn,AskReddit,"{'i': 2, 'am': 1, 'trying': 2, 'to': 6, 'set':...",114,{}
230,eusxf,gaming,"{'i': 5, ''m': 2, 'trying': 2, 'to': 4, 'get':...",145,{}
265,euswf,reddit.com,"{'welcome': 1, 'to': 4, 'www.valentinediamondc...",146,{}


In [13]:
print(main.WordCollection.vocab_to_objs)

defaultdict(<class 'list'>, {})
